### Problem Statement:

#### Build a Machine Learning model , which will help the management to Identify the potential candidates (good candidate to offer job) for both the roles (Web Development and Data Scientist) from the given data.

### Solution Approch:

* Since it is unlabbeled data, I am approching by unsuperwise Algorithms
* For Other Skill to take into the model, doing text analysis
* Splitting the Other Skill into core skills of Data Science and Web Developement
* Providing the Weight to Given skills, Other Skills and Academics
* Using K-mean clustering to make clusters

In [1]:
#importing the required Library
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
# reading data set
data=pd.read_excel('mavoix_ml_sample_dataset.xlsx')
df = data.copy() # making a copy 
df.head(3) # top 3 rows

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Deep Learning (out of 3),PHP (out of 3),MySQL (out of 3),HTML (out of 3),CSS (out of 3),JavaScript (out of 3),...,Node.js (out of 3),ReactJS (out of 3),Other skills,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
0,ML0001,Bangalore,0,2,0,2,0,2,3,2,...,0,0,R Programming,Bachelor of Science (B.Sc),Mathematics,2017,NaN,NaN,NaN,NaN
1,ML0002,Bangalore,2,0,0,2,2,2,2,2,...,0,0,"Data Science, Machine Learning, Neural Network...",Bachelor of Technology (B.Tech),Computer Science & Engineering,2019,NaN,85.50/100,92.20/92.20,96.60/96.60
2,ML0003,Bangalore,3,0,1,2,2,2,0,2,...,0,0,"Algorithms, Data Structures, Python, C Program...",Master of Science (M.Sc),Computer Science,2018,7.91/10,70.00/100,64.83/64.83,77.60/77.60


In [3]:
# data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 24 columns):
Application_ID                392 non-null object
Current City                  392 non-null object
Python (out of 3)             392 non-null int64
R Programming (out of 3)      392 non-null int64
Deep Learning (out of 3)      392 non-null int64
PHP (out of 3)                392 non-null int64
MySQL (out of 3)              392 non-null int64
HTML (out of 3)               392 non-null int64
CSS (out of 3)                392 non-null int64
JavaScript (out of 3)         392 non-null int64
Unnamed: 10                   0 non-null float64
AJAX (out of 3)               392 non-null int64
Bootstrap (out of 3)          392 non-null int64
MongoDB (out of 3)            392 non-null int64
Node.js (out of 3)            392 non-null int64
ReactJS (out of 3)            392 non-null int64
Other skills                  378 non-null object
Degree                        380 non-null object
Stream   

In [4]:
df.shape

(392, 24)

## Null Values


In [5]:
# Null value counts
df.isna().sum()

Application_ID                  0
Current City                    0
Python (out of 3)               0
R Programming (out of 3)        0
Deep Learning (out of 3)        0
PHP (out of 3)                  0
MySQL (out of 3)                0
HTML (out of 3)                 0
CSS (out of 3)                  0
JavaScript (out of 3)           0
Unnamed: 10                   392
AJAX (out of 3)                 0
Bootstrap (out of 3)            0
MongoDB (out of 3)              0
Node.js (out of 3)              0
ReactJS (out of 3)              0
Other skills                   14
Degree                         12
Stream                         18
Current Year Of Graduation      0
Performance_PG                289
Performance_UG                 54
Performance_12                163
Performance_10                177
dtype: int64

* Unnamed: 10 completely null Deleting this column
* Instead of using the stream feature, considering Skills related to Data Science and Web Devep.
* In Performance in PG/UG/12th/10th giving 1 if Score is First Devision 0 otherwise
* In Degree if they are PG then providing 1 else 0


In [6]:
# Deleting the column "Unnamed: 10"
del df['Unnamed: 10']

In [7]:
# Filling all the missing values with zero to perform the above actions
df.fillna(0,inplace = True)

#### Handling Others Skills

In [8]:
# converting type into list of string
df['Other skills']  = df['Other skills'].apply(lambda x: str(x).split(", "))

# All skill Set
skill_set = set(df['Other skills'].sum())
print(skill_set)

# Other Core Skills for Web Developement
other_core_web = {'HTML', 'SAP', 'User Interface (UI) Development', 'Search Engine Optimization (SEO)', 'Apache', 'Linux', 'Hadoop', 'Bootstrap'}

# Other Core Skills for Data Science
other_core_DataScience = {'SAS', 'Data Science', 'Natural Language Processing (NLP)', 'BIG DATA ANALYTICS', 'Statistical Modeling','Neural Network', 'Machine Learning'}

# creating two new features
df['other_DS_skills(out_6)'] = df['Other skills'].apply(lambda x: len(set(x).intersection(other_core_DataScience)))
df['other_WD_skills(out_8)'] = df['Other skills'].apply(lambda x: len(set(x).intersection(other_core_web)))

{'Angular 2.0', 'Deep Learning', 'Search Engine Marketing (SEM)', 'Hadoop', 'GNU Octave', 'Blockchain', 'Cloud Computing', 'Email Marketing', 'Dart', 'Search Engine Optimization (SEO)', 'Power BI', 'Autodesk Inventor', 'AngularJS', 'Oracle', 'Bootstrap', 'Unix', '3ds Max', 'Flask', 'Android', 'Adobe Premiere Pro', '.NET', 'R Programming', 'Google Analytics', 'Articulate Storyline', 'SolidWorks', 'MongoDB', 'jQuery', 'Hibernate (Java)', 'NI Multisim', 'Hindi Proficiency (Spoken)', 'Linux', 'English Proficiency (Written)', 'Mathematics', 'Artifical Intelligence', 'MS-Word', 'SQL', 'Computer Networks', 'Apache', 'ERP Implementation and Deployment', 'REST API', 'Punjabi Proficiency (Spoken)', 'Teaching', 'Node.js', 'AJAX', 'Telugu Proficiency (Written)', 'Adobe Flash', 'Firebase', 'Algorithms', 'MS-Excel', 'Machine Learning', 'React Native', 'Model View Controller(MVC)', 'PostgreSQL', 'Data Science', 'BIG DATA ANALYTICS', 'User Interface (UI) Development', 'C Programming', 'Bengali Profici

In [9]:
# handelling UG/PG Degree
df['Degree'] = df.Degree.apply(lambda x: str(x).split(" ")[0])
df['Degree'] = df['Degree'].replace({'B.Com.':0, 'B.Tech':0,'Bachelor':0, 'Executive':0, 'Integrated':1,
                      'MBA':1, 'Master':1, 'PG':1, 'Post':1, '0':0})

In [10]:
# functions for transforming PG/UG/12th/10th features
def division_graduation(x):
    if x != 0:
        temp = str(x).split('/')
        x = float(temp[0])
        total= int(temp[-1])
        if x/total > 0.6:
            return 1
    return 0
def performance(df,col):
    for i in col:
        df[i] = df[i].map(division_graduation)
    return df

def division_intermediate(x):
    if x > 10:
        x /= 10
    if x>6:
        return 1
    return 0
    

In [11]:
df['Performance_12'] =  df['Performance_12'].apply(lambda x:division_intermediate(float(str(x).split("/")[0])))
df['Performance_10'] =  df['Performance_10'].apply(lambda x:division_intermediate(float(str(x).split("/")[0])))

In [12]:
df = performance(df, ['Performance_PG', 'Performance_UG'])
df['year_gap'] = df['Current Year Of Graduation'] - 2020

In [13]:
# Dropping all the object columns
df = df.select_dtypes(exclude=object)

## Scores

> Providing manual Weights, 
* 70% to Given Skills
* 20% to Other Skills
* 10% to Academics

In [14]:
df['DS_SCORE'] = 0.7*(df[['Python (out of 3)','R Programming (out of 3)', 'Deep Learning (out of 3)']].\
                      sum(axis = 1)) + 0.20*(df['other_DS_skills(out_6)']) +\
                    0.10*(df[['Degree','Performance_PG', 'Performance_UG', 'Performance_12', 'Performance_10']].\
                          sum(axis = 1))

In [15]:
df['WD_SCORE'] = 0.7*(df[['PHP (out of 3)', 'MySQL (out of 3)', 'HTML (out of 3)', 'CSS (out of 3)',
                          'JavaScript (out of 3)', 'AJAX (out of 3)', 'Bootstrap (out of 3)', 
                          'MongoDB (out of 3)', 'Node.js (out of 3)','ReactJS (out of 3)']].\
                      sum(axis = 1)) + 0.20*(df['other_WD_skills(out_8)']) +\
                0.10*(df[['Degree','Performance_PG', 'Performance_UG', 'Performance_12',
                          'Performance_10']].sum(axis = 1))

## Clustering Using K-Mean Clustering

In [16]:
# create kmeans object for data science
kmeans_ds = KMeans(n_clusters=2, random_state=42)
# fit kmeans object on  data data science score
kmeans_ds.fit(np.array(df['DS_SCORE']).reshape(-1,1))
# df['DS_SCORE_cluster'] = kmeans_ds.labels_
cen_ds = kmeans_ds.cluster_centers_

In [17]:
# create kmeans object for Web Developer 
kmeans_wd = KMeans(n_clusters=2, random_state=42)
# fit kmeans clustering on Web Developer Score
kmeans_wd.fit(np.array(df['WD_SCORE']).reshape(-1,1))
cen_wd = kmeans_wd.cluster_centers_
# df['WD_SCORE_cluster'] = kmeans_wd.labels_

In [18]:
print("WD Centroid:", cen_wd)
print("DS Centroid:", cen_ds)

WD Centroid: [[6.71272727]
 [1.47709251]]
DS Centroid: [[3.48785714]
 [1.26666667]]


In [19]:
def lable(centroid, points):
    if abs(max(centroid)-points) < abs(min(centroid)-points):
        return 1
    return 0

In [20]:
df['DS_SCORE_cluster'] = df['DS_SCORE'].apply(lambda x: lable(cen_ds,x))
df['WD_SCORE_cluster'] = df['WD_SCORE'].apply(lambda x: lable(cen_wd,x))

In [21]:
data['Suitablefor'] = df[['DS_SCORE_cluster','WD_SCORE_cluster']].apply(lambda x: 1 if (x['DS_SCORE_cluster']==1) & (x['WD_SCORE_cluster']==1) else 0, axis=1)


In [22]:
def suitable(x):
    if (x.loc['DS_SCORE_cluster']==1) & (x.loc['WD_SCORE_cluster']==0):
        return 'Data Scientist'
    if (x.loc['DS_SCORE_cluster']==0) & (x.loc['WD_SCORE_cluster']==1):
        return 'Web Developer'
    if (x.loc['DS_SCORE_cluster']==1) & (x.loc['WD_SCORE_cluster']==1):
        return 'Both(Data Science & Web Developer)'
    return 'Not Eligible'
data['Suitable_for'] = df[['DS_SCORE_cluster','WD_SCORE_cluster']].apply(lambda x: suitable(x),axis =1)


#### Some Output insight

In [23]:
Candidate_for_DS_n_WD = data[data['Suitable_for'] == 'Both(Data Science & Web Developer)'].shape[0]
print("Candidate for Data Science & Web Dev both: {} out of {}".format(Candidate_for_DS_n_WD,data.shape[0]))

Candidate for Data Science & Web Dev both: 58 out of 392


In [24]:
Candidate_for_DS = data[data['Suitable_for'] == 'Data Scientist'].shape[0]
print("Candidate for Data Science: {} out of {}".format(Candidate_for_DS,data.shape[0]))

Candidate for Data Science: 82 out of 392


In [25]:
Candidate_for_WD = data[data['Suitable_for'] == 'Web Developer'].shape[0]
print("Candidate for Web Developer: {} out of {}".format(Candidate_for_WD,data.shape[0]))

Candidate for Web Developer: 107 out of 392


In [26]:
Candidate_NE = data[data['Suitable_for'] == 'Not Eligible'].shape[0]
print("Candidates that are Not Eligible: {} out of {}".format(Candidate_NE,data.shape[0]))

Candidates that are Not Eligible: 145 out of 392


In [27]:
# Saving as a dataframe
data.to_csv("Output.csv")